# 1. Import Dependencies

In [24]:
!pip install stable-baselines3[extra]
import gym 
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.4 MB/s eta 0:00:00


# 2. Load Environment

In [25]:
environment_name = "CartPole-v0"

In [26]:
env = gym.make(environment_name)

In [27]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:46.0
Episode:2 Score:26.0
Episode:3 Score:16.0
Episode:4 Score:10.0
Episode:5 Score:24.0


# 3. Understanding The Environment


In [5]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()


0

In [6]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([ 3.4063570e+00, -3.0459490e+38, -1.8623586e-01,  3.2697114e+38],
      dtype=float32)

# 4. Train an RL Model

In [7]:
log_path = os.path.join('Training','Logs')

In [8]:
log_path

'Training/Logs'

In [9]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)

Using cuda device


In [10]:
model.learn(total_timesteps=20000)

-----------------------------
| time/              |      |
|    fps             | 456  |
|    iterations      | 1    |
|    time_elapsed    | 4    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 389         |
|    iterations           | 2           |
|    time_elapsed         | 10          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007930074 |
|    clip_fraction        | 0.0947      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.0197      |
|    learning_rate        | 0.0003      |
|    loss                 | 5.42        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0139     |
|    value_loss           | 53.6        |
-----------------------------------------
----------------------------------

# 5. Save and Reload Model

In [28]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [29]:
model.save(PPO_path)

# 6. Evaluation


In [41]:
evaluate_policy(model, env, n_eval_episodes=10,)

(9.1, 0.7000000000000001)

In [40]:
env.close()

# 7. Test Model

In [42]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done: 
        print('info', info)
        break

info [{'terminal_observation': array([ 0.14251696,  1.5652064 , -0.22761227, -2.56413   ], dtype=float32)}]


# 8. Adding a callback to the training Stage

In [34]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)
model = PPO('MlpPolicy', env, verbose = 1,)
model.learn(total_timesteps=20000, callback=eval_callback)
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)
evaluate_policy(model, env, n_eval_episodes=10,)
env.close()

Using cuda device
-----------------------------
| time/              |      |
|    fps             | 839  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 598         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008302784 |
|    clip_fraction        | 0.0945      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00257    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.26        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0149     |
|    value_loss           | 56.8        |
-----------------------------------------
----------------

# 9. Changing Policies


In [37]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})
model.learn(total_timesteps=20000, callback=eval_callback)

Using cuda device
-----------------------------
| time/              |      |
|    fps             | 728  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 502         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014595163 |
|    clip_fraction        | 0.214       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | -0.00946    |
|    learning_rate        | 0.0003      |
|    loss                 | 4.02        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0237     |
|    value_loss           | 17.9        |
-----------------------------------------
----------------

# 10. Using an Alternate Algorithm

In [38]:
from stable_baselines3 import DQN
model = DQN('MlpPolicy', env, verbose = 1,)
model.learn(total_timesteps=20000, callback=eval_callback)
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')
model.save(dqn_path)
model = DQN.load(dqn_path, env=env)
evaluate_policy(model, env, n_eval_episodes=10,)
env.close()

Using cuda device
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.953    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 9599     |
|    time_elapsed     | 0        |
|    total_timesteps  | 98       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.923    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 9077     |
|    time_elapsed     | 0        |
|    total_timesteps  | 163      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.884    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 8875     |
|    time_elapsed     | 0        |
|    total_timesteps  | 245      |
----------------------------------
----------------------------------
| 